In [1]:
pip install hyperopt

Note: you may need to restart the kernel to use updated packages.


In [13]:
!tensorboard --logdir C:/Users/Naoya/Master_thesis/my_log_dir2

^C


# 実験参加者別

# 隠れ層のチューニングあり

In [1]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, Trials, fmin
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import TensorBoard

#ニューロン数を20にするなどしてパラメータ数を多くするとALLATRIBUTEERRORが発生する
#hp.choiceを使ったパラメータ値のprint(best)による出力は、選択した数値ではなくあくまでインデックスの値を返すため+1する必要がある
hyperopt_parameters = {
    'n_hidden': hp.choice('n_hidden',[1, 2, 3, 4, 5]),
    #'n_neurons': hp.choice('n_neurons', [[hp.quniform(f'n_neurons_{neuron}', 1, 20, 1) for neuron in range(10)]]),
    'n_neurons': hp.choice('n_neurons', [[hp.choice(f'n_neurons_{neuron}',
                                                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) for neuron in range(10)]]),
    'n_drop': hp.choice('n_drop', [[hp.uniform(f'n_drop_{drop}', 0.3, 0.5) for drop in range(10)]]),
    'optimizer': hp.choice('optimizer', ['Adam', 'SGD', 'RMSprop']),
    'batch_size': hp.choice('batch_size', [8, 16, 32, 64])
    #'epochs': hp.choice('epochs', [100, 250, 500, 1000, 2000]),
    #'learning_rate': hp.uniform('learning_rate', 0.00001, 0.01),
}

# 静止状態

df = pd.read_csv('okada_K2.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:,:6]
y_train = df.iloc[:,6]
y_mean = np.mean(y_train)
y_std = np.std(y_train)
y_train = (y_train - y_mean)/y_std
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=0)

def function(args):
    #n_hidden = int(args['n_hidden'])
    n_hidden = args['n_hidden']
    n_drop = args['n_drop']
    batch_size = int(args['batch_size'])
    optimizer = args['optimizer']
    n_neurons = args['n_neurons']
    #print(args['n_hidden'],args['n_neurons'],args['n_drop'],args['batch_size'],args['optimizer'])
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(6,)))
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons[layer], activation='relu'))
        model.add(layers.Dropout(n_drop[layer]))
        #print(args['nneurons'][layer],args['ndrop'][layer])
    model.add(layers.Dense(1, activation='linear'))
    #model.summary()
    
    model.compile(optimizer=optimizer, loss='mse', metrics='mae')

    callbacks = [EarlyStopping(monitor = 'val_mae', patience = 20)]
    
    model.fit(X_train,
              y_train,
              epochs=1000,
              batch_size=batch_size,
              verbose=0,
              callbacks=callbacks,
              validation_split=0.1) # 検証用のデータ
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(evaluation[0])
    return evaluation[0]

#for i in tqdm(range(200)):
 #   trials = Trials()
  #  best = fmin(function,
   #             hyperopt_parameters,
    #            algo=tpe.suggest,
     #           max_evals=1,
      #          trials=trials,
       #         verbose=0)
    
trials = Trials()
best = fmin(function,
            hyperopt_parameters,
            algo=tpe.suggest,
            max_evals=200,
            trials=trials,
            verbose=0)

1.0453646183013916
0.8998081088066101
1.1225701570510864
1.068488359451294
1.2261911630630493
1.143628478050232
1.107463002204895
1.084760069847107
1.1520825624465942
1.1602040529251099
0.7829303741455078
1.078192114830017
1.1640106439590454
0.6379026770591736
1.1055513620376587
0.760429859161377
1.064826250076294
1.1433197259902954
1.0801656246185303
1.2221111059188843
0.7531495094299316
0.7526265978813171
0.7508746981620789
0.8104107975959778
0.7637422680854797
0.8049700856208801
0.8347452282905579
0.726554811000824
1.0134295225143433
0.811396062374115
1.020084023475647
1.1384034156799316
1.1461492776870728
0.8772363066673279
0.7337151169776917
1.1438242197036743
0.987175703048706
1.1398696899414062
1.140297293663025
0.7772789001464844
1.1409085988998413
0.9923028945922852
0.7287809252738953
1.141053557395935
0.761354923248291
1.1504058837890625
1.14142644405365
1.1419885158538818
0.958526611328125
0.7884743809700012
1.1537264585494995
0.964660108089447
1.1247252225875854
0.919652938

In [2]:
print(best)

{'batch_size': 1, 'n_drop': 0, 'n_drop_0': 0.33974299308471934, 'n_drop_1': 0.4821875483076665, 'n_drop_2': 0.355426805273229, 'n_drop_3': 0.49622233764912715, 'n_drop_4': 0.3601546899222607, 'n_drop_5': 0.35873674925518034, 'n_drop_6': 0.430105685827313, 'n_drop_7': 0.47230736500813225, 'n_drop_8': 0.41512331802483937, 'n_drop_9': 0.47198284431598236, 'n_hidden': 0, 'n_neurons': 0, 'n_neurons_0': 7, 'n_neurons_1': 7, 'n_neurons_2': 9, 'n_neurons_3': 0, 'n_neurons_4': 0, 'n_neurons_5': 9, 'n_neurons_6': 7, 'n_neurons_7': 0, 'n_neurons_8': 0, 'n_neurons_9': 8, 'optimizer': 1}


In [10]:
#5
print(best)

{'batch_size': 2, 'n_drop': 0, 'n_drop_0': 0.38991779865074466, 'n_drop_1': 0.4134436274982293, 'n_drop_2': 0.3417495990777167, 'n_drop_3': 0.4218835814717209, 'n_drop_4': 0.4850267814094251, 'n_drop_5': 0.4181478551178712, 'n_drop_6': 0.30012623173534975, 'n_drop_7': 0.4876931890614151, 'n_drop_8': 0.45699944571371287, 'n_drop_9': 0.3999050494456673, 'n_hidden': 0, 'n_neurons': 0, 'n_neurons_0': 7, 'n_neurons_1': 2, 'n_neurons_2': 6, 'n_neurons_3': 9, 'n_neurons_4': 8, 'n_neurons_5': 2, 'n_neurons_6': 8, 'n_neurons_7': 4, 'n_neurons_8': 7, 'n_neurons_9': 0, 'optimizer': 1}


In [8]:
#4
print(best)

{'batch_size': 2, 'n_drop': 0, 'n_drop_0': 0.3122794447292501, 'n_drop_1': 0.3957052878635843, 'n_drop_2': 0.35618021983586834, 'n_drop_3': 0.3714288601839902, 'n_drop_4': 0.3183172877715252, 'n_drop_5': 0.4854759357005794, 'n_drop_6': 0.3451761068305333, 'n_drop_7': 0.4873889650813854, 'n_drop_8': 0.492007451771217, 'n_drop_9': 0.42836835030764514, 'n_hidden': 1, 'n_neurons': 0, 'n_neurons_0': 9, 'n_neurons_1': 9, 'n_neurons_2': 2, 'n_neurons_3': 9, 'n_neurons_4': 3, 'n_neurons_5': 8, 'n_neurons_6': 4, 'n_neurons_7': 7, 'n_neurons_8': 8, 'n_neurons_9': 4, 'optimizer': 1}


In [6]:
#3
print(best)

{'batch_size': 2, 'n_drop': 0, 'n_drop_0': 0.32390912297135954, 'n_drop_1': 0.42748606109032344, 'n_drop_2': 0.44429212718387334, 'n_drop_3': 0.4190692923066496, 'n_drop_4': 0.36459130463963446, 'n_drop_5': 0.3963518656092614, 'n_drop_6': 0.32866071240971373, 'n_drop_7': 0.41384258180894273, 'n_drop_8': 0.32998520487995453, 'n_drop_9': 0.33388615576929814, 'n_hidden': 0, 'n_neurons': 0, 'n_neurons_0': 9, 'n_neurons_1': 7, 'n_neurons_2': 9, 'n_neurons_3': 9, 'n_neurons_4': 8, 'n_neurons_5': 6, 'n_neurons_6': 3, 'n_neurons_7': 0, 'n_neurons_8': 3, 'n_neurons_9': 0, 'optimizer': 1}


In [4]:
#2
print(best)

{'batch_size': 0, 'n_drop': 0, 'n_drop_0': 0.3188286882828955, 'n_drop_1': 0.3041299943182385, 'n_drop_2': 0.3687489519724732, 'n_drop_3': 0.40700468002024565, 'n_drop_4': 0.4054258787410093, 'n_drop_5': 0.3442792676492256, 'n_drop_6': 0.4337065714422552, 'n_drop_7': 0.44967762323274013, 'n_drop_8': 0.3317729020093095, 'n_drop_9': 0.35911659800899554, 'n_hidden': 0, 'n_neurons': 0, 'n_neurons_0': 8, 'n_neurons_1': 3, 'n_neurons_2': 4, 'n_neurons_3': 2, 'n_neurons_4': 2, 'n_neurons_5': 9, 'n_neurons_6': 0, 'n_neurons_7': 5, 'n_neurons_8': 4, 'n_neurons_9': 8, 'optimizer': 1}


In [2]:
#1
print(best)

{'batch_size': 1, 'n_drop': 0, 'n_drop_0': 0.3002234397708743, 'n_drop_1': 0.3873031913493352, 'n_drop_2': 0.4190518150668984, 'n_drop_3': 0.3733095204711307, 'n_drop_4': 0.41331600553890485, 'n_drop_5': 0.3694304684383882, 'n_drop_6': 0.34437744253082014, 'n_drop_7': 0.3744783198439581, 'n_drop_8': 0.31232261743813067, 'n_drop_9': 0.34645204966275506, 'n_hidden': 0, 'n_neurons': 0, 'n_neurons_0': 4, 'n_neurons_1': 8, 'n_neurons_2': 0, 'n_neurons_3': 5, 'n_neurons_4': 9, 'n_neurons_5': 4, 'n_neurons_6': 5, 'n_neurons_7': 1, 'n_neurons_8': 2, 'n_neurons_9': 5, 'optimizer': 0}


In [ ]:
# 最適なハイパーパラメータを使用して再学習

num_layers = int(best_params[0])
neurons_per_layer = [int(best_params[i]) for i in range(1, num_layers+1)]
dropout_rate = best_params[num_layers+1]
batch_size = int(best_params[num_layers+2])
optimizer = ['adam', 'sgd', 'rmsprop'][int(best_params[num_layers+3])]

model = models.Sequential()
model.add(layers.Dense(neurons_per_layer[0], activation='relu', input_shape=(6,)))
for i in range(1, num_layers):
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(neurons_per_layer[i], activation='relu'))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae'])

callbacks = [EarlyStopping(monitor='val_mae', patience=20)]

history = model.fit(x=X_train,
                    y=y_train,
                    epochs=500,
                    batch_size=batch_size,
                    verbose=0,
                    callbacks=callbacks,
                    validation_split=0.1)

score = model.evaluate(X_test, y_test)
predictions = model.predict(X_test).flatten()
print("Test set loss: {}, Test set mean_squared_error: {}".format(score[0], score[1]))
print(predictions)

In [ ]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, Trials, fmin
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import TensorBoard

#ニューロン数を20にするなどしてパラメータ数を多くするとALLATRIBUTEERRORが発生する
#hp.choiceを使ったパラメータ値のprint(best)による出力は、選択した数値ではなくあくまでインデックスの値を返すため+1する必要がある
hyperopt_parameters = {
    'n_hidden': hp.choice('n_hidden',[1, 2, 3, 4, 5]),
    #'n_neurons': hp.choice('n_neurons', [[hp.quniform(f'n_neurons_{neuron}', 1, 20, 1) for neuron in range(10)]]),
    'n_neurons': hp.choice('n_neurons', [[hp.choice(f'n_neurons_{neuron}',
                                                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) for neuron in range(10)]]),
    'n_drop': hp.choice('n_drop', [[hp.uniform(f'n_drop_{drop}', 0.3, 0.5) for drop in range(10)]]),
    'optimizer': hp.choice('optimizer', ['Adam', 'SGD', 'RMSprop']),
    'batch_size': hp.choice('batch_size', [8, 16, 32, 64])
    #'epochs': hp.choice('epochs', [100, 250, 500, 1000, 2000]),
    #'learning_rate': hp.uniform('learning_rate', 0.00001, 0.01),
}

# 静止状態

df = pd.read_csv('okada_K2.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:,:6]
y_train = df.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=0)

def function(args):
    #n_hidden = int(args['n_hidden'])
    n_hidden = args['n_hidden']
    n_drop = args['n_drop']
    batch_size = int(args['batch_size'])
    optimizer = args['optimizer']
    n_neurons = args['n_neurons']
    #print(args['n_hidden'],args['n_neurons'],args['n_drop'],args['batch_size'],args['optimizer'])
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(6,)))
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons[layer], activation='relu'))
        model.add(layers.Dropout(n_drop[layer]))
        #print(args['nneurons'][layer],args['ndrop'][layer])
    model.add(layers.Dense(1, activation='linear'))
    #model.summary()
    
    model.compile(optimizer=optimizer, loss='mse', metrics='mae')

    callbacks = [EarlyStopping(monitor = 'val_mae', patience = 20)]
    
    model.fit(X_train,
              y_train,
              epochs=1000,
              batch_size=batch_size,
              verbose=0,
              callbacks=callbacks,
              validation_split=0.1) # 検証用のデータ
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    return evaluation[0]

#for i in tqdm(range(200)):
 #   trials = Trials()
  #  best = fmin(function,
   #             hyperopt_parameters,
    #            algo=tpe.suggest,
     #           max_evals=1,
      #          trials=trials,
       #         verbose=0)
    
trials = Trials()
best = fmin(function,
            hyperopt_parameters,
            algo=tpe.suggest,
            max_evals=200,
            trials=trials,
            verbose=0)

# 隠れ層のチューニングなし

In [4]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, Trials, fmin
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tqdm.notebook import tqdm

hyperopt_parameters = {
    'l1_drop': hp.uniform('l1_drop', 0.3, 0.5),
    'l2_drop': hp.uniform('l2_drop', 0.3, 0.5),
    'l1_out': hp.choice('l1_out',[1, 3, 5, 10, 15, 20]),
    'l2_out': hp.choice('l2_out',[1, 3, 5, 10, 15, 20]),
    'optimizer': hp.choice('optimizer', ['Adam', 'SGD', 'RMSprop','Adagrad','Adadelta']),
    'batch_size': hp.choice('batch_size', [8, 16, 32, 64]),
    'epochs': hp.choice('epochs', [100, 250, 500, 1000, 2000]),
    #'learning_rate': hp.uniform('learning_rate', 0.00001, 0.01),
}

# 静止状態

df = pd.read_csv('okada_K3.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:,:6]
y_train = df.iloc[:,6]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=0)

def function(args):
    model = models.Sequential()
    model.add(layers.Dense(args['l1_out'], activation='relu', input_shape=(6,)))
    model.add(layers.Dropout(args['l1_drop']))
    model.add(layers.Dense(args['l2_out'], activation='relu'))
    model.add(layers.Dropout(args['l2_drop']))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(optimizer=args['optimizer'], loss='mse', metrics='mse')

    callbacks = [EarlyStopping(monitor = 'val_mse', patience = 20)]

    model.fit(X_train,
              y_train,
              epochs=args['epochs'],
              batch_size=args['batch_size'],
              verbose=0,
              callbacks=callbacks,
              validation_split=0.1) # 検証用のデータ
    
    evaluation = model.evaluate(X_test, y_test, batch_size=args['batch_size'], verbose=0)
    return evaluation[0]

#trials = Trials()
#best = fmin(function,
 #               hyperopt_parameters,
  #              algo=tpe.suggest,
   #             max_evals=200,
    #            trials=trials,
     #           verbose=0)
for i in tqdm(range(200)):
    trials = Trials()
    best = fmin(function,
                hyperopt_parameters,
                algo=tpe.suggest,
                max_evals=1,
                trials=trials,
                verbose=0)

In [2]:
print(best)

{'batch_size': 0, 'epochs': 4, 'l1_drop': 0.2228814672471817, 'l1_out': 3, 'l2_drop': 0.001771712353763788, 'l2_out': 1, 'optimizer': 0}


In [8]:
best['batch_size']

3

# GP-EIによる最適化

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from GPyOpt.methods import BayesianOptimization
from tqdm.notebook import tqdm

# データの読み込み
df = pd.read_csv('RUD_K_okada2_test3.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:, :4]
y_train = df.iloc[:, 5]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

# 目的関数の定義
def function(parameters):
    print(parameters)
    n_hidden = int(parameters[0, 0])
    #n_drop = parameters[0, 1]
    batch_size = int(parameters[0, 1])
    optimizer = ['adam', 'sgd', 'rmsprop'][int(parameters[0, 2])]
    #n_neurons = parameters[0, 4:n_hidden+4].astype(int)
    n_neurons = parameters[0, 3:13].astype(int)
    n_drop = parameters[0, 13:23]
   # print(n_hidden,n_neurons, n_drop,batch_size,optimizer)
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(4,)))
    for i in range(n_hidden):
        model.add(layers.Dense(n_neurons[i], activation='relu'))
        #leaky_relu
        model.add(layers.Dropout(n_drop[i]))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])

    callbacks = [EarlyStopping(monitor='val_mae', patience=20)]

    history = model.fit(x=X_train,
                        y=y_train,
                        epochs=2000,
                        batch_size=batch_size,
                        verbose=0,
                        callbacks=callbacks,
                        validation_split=0.1)
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    #print(evaluation[0])
    return evaluation[0]
    #score = model.evaluate(X_test, y_test)
    #predictions = predict_function(model, X_test).flatten()
    #predictions = model.predict(X_test).flatten()
    #mse = mean_squared_error(y_test, predictions)
    #return -mse

# ハイパーパラメータの範囲とタイプの指定
#max_layers = 10  # チューニングする最大層数
bounds = [{'name': 'n_hidden', 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)},
          #{'name': 'n_drop', 'type': 'continuous', 'domain': (0.3, 0.5)},
          {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32, 64)},
          {'name': 'optimizer', 'type': 'discrete', 'domain': (0, 1, 2)}]
for i in range(1, 11):
    bounds += [{'name': 'n_neurons_{}'.format(i), 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                                                 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)}]
for j in range(1, 11):
    bounds += [{'name': 'n_drop_{}'.format(j), 'type': 'continuous', 'domain': (0.3, 0.5)}]
# ベイズ最適化の設定
myRelt = BayesianOptimization(f=function,
                              domain=bounds,
                              #initinal_design_numdata=5,
                              acquisition_type='EI')
myRelt.run_optimization(max_iter=200)

# チューニング結果の取得
X = myRelt.X  # パラメータの履歴
Y = myRelt.Y  # 目的関数の値の履歴

print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)    #MSE

#myRelt.plot_acquisition()
#myRelt.plot_convergence()


###最適なハイパーパラメータとそのときのMSEを表示
#best_params = myRelt.X[np.argmin(myRelt.Y)]
#best_mse = -np.min(myRelt.Y)
#print("Best Parameters:", best_params)
#print("Best MSE:", best_mse)

[[ 6.         16.          0.         15.         16.         11.
  15.          3.         11.          7.         11.         12.
   7.          0.46146048  0.31366683  0.33579516  0.41215964  0.44134846
   0.36844898  0.36150442  0.4870921   0.36594821  0.32314823]]
[[ 4.          8.          1.          7.          8.          6.
   7.         10.          8.         13.          6.          1.
  17.          0.33131622  0.4032712   0.40289095  0.34169084  0.36344491
   0.38637462  0.32798254  0.47208793  0.48940796  0.38899528]]
[[ 5.         16.          2.          5.         16.          3.
  17.         10.         10.         20.         14.         10.
  12.          0.35242363  0.45360033  0.44789597  0.32710021  0.42733789
   0.41679138  0.41713805  0.3239864   0.49788336  0.34027487]]
[[ 6.         16.          0.          5.          6.         17.
  20.         13.         17.         13.          9.         17.
   1.          0.43437289  0.31686813  0.49095937  0.33081

[[ 8.         32.          0.          1.          1.         11.
   8.         20.         11.          5.         11.         16.
  17.          0.31950726  0.48102604  0.44075471  0.35952177  0.38337483
   0.40468365  0.49417307  0.33790268  0.4757302   0.49429023]]
[[ 6.         16.          0.         17.         18.          5.
  15.         13.         11.          8.         18.         18.
  20.          0.43870324  0.40591472  0.49187831  0.44948995  0.403175
   0.34152567  0.32521441  0.47810755  0.41515514  0.46412171]]
[[ 1.         64.          1.         14.         15.          9.
  18.          8.          4.          9.         13.         19.
   6.          0.49276745  0.4720263   0.33487903  0.34741459  0.44854081
   0.45733181  0.38716421  0.40037406  0.39720467  0.44099664]]
[[ 5.         16.          1.         17.          9.         18.
   7.         19.         20.          4.          6.         13.
  13.          0.33366038  0.46769791  0.48841713  0.3385888

[[ 1.          8.          1.          8.          8.          1.
  17.          3.         17.         16.         12.         14.
  11.          0.37042979  0.39547868  0.34830204  0.4364368   0.4600835
   0.49960011  0.48140952  0.32480261  0.35420315  0.37123211]]
[[ 8.         64.          0.         10.          1.          1.
  12.          4.          1.         10.          6.         12.
   4.          0.45117267  0.43582292  0.49495964  0.4001037   0.38727777
   0.47443988  0.40442428  0.49767732  0.41383096  0.31475171]]
[[ 8.         32.          2.         11.          2.         12.
   3.         12.         16.         17.         14.         18.
   1.          0.48207582  0.42362533  0.46129457  0.41036657  0.40772816
   0.44153633  0.36067047  0.47951722  0.43480124  0.40058464]]
[[ 6.         16.          1.         16.          6.         11.
   4.          3.          7.          9.         15.         19.
   5.          0.32107676  0.44602667  0.38333628  0.366002

[[10.         64.          1.         10.         20.         20.
   3.         12.          9.          2.         13.         16.
   8.          0.31701925  0.40546975  0.41374062  0.39191928  0.35064109
   0.35759407  0.44787623  0.39095702  0.35278022  0.37904823]]
[[ 2.         64.          1.         18.         10.         14.
   7.         15.          5.         13.          8.         16.
   9.          0.3801545   0.3481501   0.48100508  0.44866042  0.43956821
   0.38467363  0.36888578  0.41139367  0.33121761  0.41338716]]
[[ 6.          8.          2.         20.         18.          7.
  12.          2.         17.         11.         16.          2.
   8.          0.3200751   0.39301699  0.31574815  0.37140613  0.39241529
   0.48784993  0.35090705  0.47950397  0.4510796   0.36815311]]
[[ 8.         16.          0.          6.         19.         16.
  12.          6.          5.         16.          6.          3.
  10.          0.30244805  0.47174704  0.4489901   0.35132

[[ 1.          8.          0.         18.         10.         10.
   1.         20.         16.         17.         19.         13.
  16.          0.30668159  0.30896022  0.37474547  0.4780972   0.41134563
   0.45104064  0.30391171  0.48602654  0.32521619  0.47332677]]
[[ 3.         32.          2.         14.          1.         10.
  18.         18.          6.         20.         18.         10.
  15.          0.37949932  0.4885714   0.34745821  0.4390041   0.33415655
   0.43927343  0.45979252  0.36344599  0.39616703  0.41692391]]
[[ 1.          8.          1.          8.         16.          3.
   3.         11.          9.          8.          3.         14.
   5.          0.47377378  0.40735177  0.32177867  0.41765107  0.4166536
   0.44698444  0.30698033  0.47517748  0.43155299  0.35277197]]
[[ 5.         64.          0.          5.          2.         15.
   3.         15.          8.         16.          7.          7.
  11.          0.42425228  0.30227356  0.33023775  0.309552

[[ 2.         32.          1.          8.         16.          5.
   8.          7.          1.          8.         11.          6.
  17.          0.49457644  0.42634349  0.34355385  0.32750477  0.33769752
   0.45733197  0.36497217  0.30292545  0.30881128  0.30701157]]
[[ 4.         16.          1.         19.          4.         15.
  16.          2.          1.         20.         14.         17.
  13.          0.49837855  0.33500207  0.40988081  0.45339225  0.35843882
   0.45862349  0.41693677  0.45883271  0.33819584  0.30235886]]
[[ 4.          8.          0.          7.         20.         15.
  19.         18.         14.          9.         11.         19.
  18.          0.30411055  0.48628203  0.42100914  0.43113101  0.42512364
   0.48404284  0.46321957  0.37966136  0.48629693  0.41904844]]
[[ 2.         64.          2.         16.         11.         10.
   6.          9.         18.          3.         10.          6.
  17.          0.4199945   0.48267524  0.48202242  0.33921

[[ 2.         32.          1.          6.         14.         16.
   9.          4.          2.         10.         12.         15.
  20.          0.47268259  0.39389691  0.36796363  0.39170985  0.36460834
   0.46023164  0.4757596   0.31542198  0.4340788   0.42361643]]
[[ 4.         64.          1.         15.         13.          7.
  15.         13.         14.          4.         13.          8.
   7.          0.32773157  0.34854989  0.36152363  0.47132159  0.33516384
   0.30225435  0.45215529  0.32254674  0.39155831  0.4861717 ]]
[[ 2.         16.          1.         17.         13.          7.
  11.          1.         13.          7.         13.         20.
  10.          0.39678198  0.38191086  0.38624308  0.36179424  0.49804183
   0.34704251  0.46389364  0.34387385  0.41407432  0.42812681]]
[[ 3.          8.          2.         19.         15.          8.
  19.          1.          3.          4.          7.          6.
   7.          0.30504246  0.41027829  0.4384905   0.31383

In [22]:
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 4.          8.          1.          7.          8.          6.
  7.         10.          8.         13.          6.          1.
 17.          0.33131622  0.4032712   0.40289095  0.34169084  0.36344491
  0.38637462  0.32798254  0.47208793  0.48940796  0.38899528]
nan


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from GPyOpt.methods import BayesianOptimization
from tqdm.notebook import tqdm

# データの読み込み
df = pd.read_csv('RUD_K_okada2_test3.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:, :4]
y_train = df.iloc[:, 4]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

# 目的関数の定義
def function(parameters):
    print(parameters)
    n_hidden = int(parameters[0, 0])
    #n_drop = parameters[0, 1]
    batch_size = int(parameters[0, 1])
    optimizer = ['adam', 'rmsprop'][int(parameters[0, 2])]
    #n_neurons = parameters[0, 4:n_hidden+4].astype(int)
    n_neurons = parameters[0, 3:13].astype(int)
    n_drop = parameters[0, 13:23]
   # print(n_hidden,n_neurons, n_drop,batch_size,optimizer)
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(4,)))
    for i in range(n_hidden):
        model.add(layers.Dense(n_neurons[i], activation='relu'))
        #leaky_relu
        model.add(layers.Dropout(n_drop[i]))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])

    callbacks = [EarlyStopping(monitor='val_mae', patience=20)]

    history = model.fit(x=X_train,
                        y=y_train,
                        epochs=2000,
                        batch_size=batch_size,
                        verbose=0,
                        callbacks=callbacks,
                        validation_split=0.1)
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    #print(evaluation[0])
    return evaluation[0]
    #score = model.evaluate(X_test, y_test)
    #predictions = predict_function(model, X_test).flatten()
    #predictions = model.predict(X_test).flatten()
    #mse = mean_squared_error(y_test, predictions)
    #return -mse

# ハイパーパラメータの範囲とタイプの指定
#max_layers = 10  # チューニングする最大層数
bounds = [{'name': 'n_hidden', 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)},
          #{'name': 'n_drop', 'type': 'continuous', 'domain': (0.3, 0.5)},
          {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32, 64)},
          {'name': 'optimizer', 'type': 'discrete', 'domain': (0, 1)}]
for i in range(1, 11):
    bounds += [{'name': 'n_neurons_{}'.format(i), 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                                                 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)}]
for j in range(1, 11):
    bounds += [{'name': 'n_drop_{}'.format(j), 'type': 'continuous', 'domain': (0.3, 0.5)}]
# ベイズ最適化の設定
myRelt = BayesianOptimization(f=function,
                              domain=bounds,
                              #initinal_design_numdata=5,
                              acquisition_type='EI')
myRelt.run_optimization(max_iter=200)

# チューニング結果の取得
X = myRelt.X  # パラメータの履歴
Y = myRelt.Y  # 目的関数の値の履歴

print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)    #MSE

#myRelt.plot_acquisition()
#myRelt.plot_convergence()


###最適なハイパーパラメータとそのときのMSEを表示
#best_params = myRelt.X[np.argmin(myRelt.Y)]
#best_mse = -np.min(myRelt.Y)
#print("Best Parameters:", best_params)
#print("Best MSE:", best_mse)

[[ 7.         64.          1.         12.         14.         11.
  12.          7.         11.         12.         18.         17.
  12.          0.36586361  0.43034085  0.31161965  0.40284859  0.37961416
   0.32211273  0.44998657  0.3242915   0.45697603  0.40539276]]
[[ 8.         16.          1.          3.          4.          8.
   8.         19.         14.          5.         11.          9.
  18.          0.40152856  0.4232649   0.34757546  0.40862891  0.33117228
   0.45187495  0.47751985  0.4490898   0.31755843  0.36641201]]
[[ 2.         32.          0.          2.         13.          3.
   1.         19.         13.         13.          5.         10.
  10.          0.38604442  0.31880358  0.38403581  0.4602604   0.42780384
   0.39863789  0.40473458  0.35258477  0.37103598  0.36852921]]
[[ 3.          8.          1.         10.         12.          6.
   6.         20.          8.          6.         17.          6.
   3.          0.46472527  0.31579581  0.38272169  0.40954

[[ 1.         32.          0.         20.         20.         18.
   1.         20.          1.          5.         14.          3.
   7.          0.38979149  0.5         0.37984584  0.428341    0.3
   0.5         0.3         0.49682891  0.5         0.3       ]]
[[ 1.         16.          0.         17.         14.          1.
   1.          1.         12.         15.          9.          1.
   7.          0.44330385  0.5         0.3         0.49769602  0.3
   0.5         0.3         0.5         0.5         0.30167509]]
[[ 1.         32.          0.         20.         20.         20.
   5.          1.          3.          6.         16.         12.
  18.          0.37275719  0.4761145   0.5         0.3239736   0.3
   0.5         0.3         0.5         0.42179335  0.3       ]]
[[ 1.         32.          0.         20.         20.         20.
   7.          1.         12.          6.         20.          1.
   8.          0.5         0.49932156  0.5         0.5         0.3
   0.5      

[[ 1.          8.          0.         20.         20.         20.
  20.         20.         20.         20.         20.          1.
   1.          0.3         0.5         0.5         0.5         0.49813659
   0.5         0.3         0.5         0.5         0.3       ]]
[[ 1.  16.   0.  20.  20.  19.   1.   1.  20.   1.  20.   5.   7.   0.5
   0.5  0.3  0.5  0.3  0.5  0.5  0.3  0.5  0.3]]
[[ 1.  32.   0.  20.  20.   1.   1.  16.  20.   1.   7.  15.  20.   0.3
   0.5  0.5  0.5  0.3  0.5  0.3  0.5  0.5  0.5]]
[[ 1.   8.   0.  20.  20.  20.  20.  20.  20.  20.  14.   1.  20.   0.3
   0.5  0.5  0.5  0.5  0.5  0.3  0.5  0.5  0.5]]
[[ 1.   8.   0.  20.   1.  20.  20.  20.  20.   1.  20.   1.   4.   0.3
   0.5  0.5  0.3  0.5  0.5  0.5  0.5  0.5  0.5]]
[[ 1.   8.   0.  20.  20.   1.  20.  20.  20.  20.  20.   1.  20.   0.3
   0.5  0.5  0.3  0.5  0.5  0.3  0.5  0.5  0.5]]
[[ 1.  16.   0.  20.   1.   1.  20.  20.  20.  20.  20.   1.   1.   0.3
   0.5  0.5  0.5  0.3  0.5  0.3  0.5  0.5  0.5]]
[[ 1

[[ 1.  32.   1.  20.   1.   1.  20.  20.  20.  20.   1.   1.   1.   0.3
   0.3  0.5  0.3  0.3  0.5  0.3  0.5  0.3  0.5]]
[[ 1.   8.   1.  20.  20.  10.   1.   1.  20.  20.  12.  20.  20.   0.3
   0.5  0.5  0.3  0.5  0.5  0.5  0.5  0.3  0.5]]
[[ 1.   8.   1.  20.   1.   1.   1.  20.  20.  20.  20.  20.   1.   0.3
   0.5  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.  32.   1.  20.   1.   1.   1.  20.   1.  20.   1.  20.   1.   0.3
   0.5  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.   8.   1.  20.  20.   4.   1.  20.  20.  20.  14.  20.  20.   0.3
   0.5  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.  16.   1.  20.   1.   1.   1.   1.  20.  20.  17.  20.   7.   0.3
   0.5  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.   8.   1.  20.   1.   1.  20.  20.  20.   1.   1.  20.  20.   0.3
   0.3  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.   8.   1.  20.   1.   1.  20.   1.  20.  20.   1.  20.  20.   0.3
   0.3  0.5  0.3  0.3  0.5  0.5  0.5  0.3  0.5]]
[[ 1.  16.   0.   6.   7.   1.  

In [26]:
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.   8.   0.  20.  20.  20.  20.  20.  20.   1.  20.   1.  20.   0.3
  0.5  0.5  0.3  0.3  0.5  0.3  0.5  0.5  0.3]
369.692138671875


In [16]:
#関節角度
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.  64.   0.  20.  20.   8.  20.  20.  20.   1.  15.   1.  20.   0.3
  0.5  0.5  0.5  0.5  0.3  0.5  0.3  0.5  0.3]
1.851058006286621


In [12]:
#J_test4
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.  16.   2.   4.  17.   1.  20.   6.   1.  20.  20.   5.   1.   0.3
  0.3  0.3  0.3  0.5  0.3  0.3  0.3  0.5  0.5]
1.9939818382263184


In [9]:
#J_test4
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 2.          8.          1.         17.          7.         11.
 14.          9.          6.          5.          2.          3.
  9.          0.31260456  0.36782765  0.47810552  0.34235326  0.47031869
  0.4469557   0.45948776  0.4922093   0.4158464   0.39217474]
3.50406551361084


In [2]:
#J_test3
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 7.         32.          0.          1.          7.          5.
  1.         10.          5.          7.          0.33866105  0.5
  0.3         0.3         0.3         0.3         0.44535237]
9.812644030660067e-09


In [6]:
#J_test2
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 7.  16.   0.   7.   1.   4.   5.   6.   1.   8.   0.3  0.3  0.5  0.3
  0.3  0.3  0.5]
9.848389659339318e-09


In [4]:
#J_test
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 7.  16.   2.   1.   9.   2.  10.   5.  10.   7.   0.5  0.3  0.3  0.3
  0.3  0.3  0.3]
9.81240066977307e-09


In [12]:
#K
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.         16.          1.         10.          6.          1.
  1.         10.         10.          2.          0.5         0.5
  0.3         0.46716815  0.5         0.3         0.40617582]
2.0344951152801514


In [10]:
#K
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.         32.          1.          6.         10.          4.
  3.          3.          5.          2.          0.47557927  0.45322611
  0.45634483  0.37157877  0.38722204  0.45363192  0.31898197]
2.1508476734161377


In [8]:
#D
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[ 1.   8.   2.  10.   7.  10.   8.   2.   4.   1.   0.5  0.5  0.3  0.3
  0.3  0.3  0.3]
0.0002846872666850686


In [6]:
#J
print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)  

[4.         8.         2.         3.         1.         5.
 3.         1.         7.         2.         0.40331947 0.33508588
 0.35694392 0.34865551 0.39585452 0.3        0.40922117]
6.28968654936557e-09


# 実験参加者共有

# 隠れ層のチューニングあり

In [ ]:
import pandas as pd
import numpy as np

from hyperopt import hp, tpe, Trials, fmin
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import TensorBoard

#ニューロン数を20にするなどしてパラメータ数を多くするとALLATRIBUTEERRORが発生する
#hp.choiceを使ったパラメータ値のprint(best)による出力は、選択した数値ではなくあくまでインデックスの値を返すため+1する必要がある
hyperopt_parameters = {
    'n_hidden': hp.choice('n_hidden',[1, 2, 3, 4, 5]),
    #'n_neurons': hp.choice('n_neurons', [[hp.quniform(f'n_neurons_{neuron}', 1, 20, 1) for neuron in range(10)]]),
    'n_neurons': hp.choice('n_neurons', [[hp.choice(f'n_neurons_{neuron}',
                                                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) for neuron in range(10)]]),
    'n_drop': hp.choice('n_drop', [[hp.uniform(f'n_drop_{drop}', 0.3, 0.5) for drop in range(10)]]),
    'optimizer': hp.choice('optimizer', ['Adam', 'SGD', 'RMSprop']),
    'batch_size': hp.choice('batch_size', [8, 16, 32, 64])
    #'epochs': hp.choice('epochs', [100, 250, 500, 1000, 2000]),
    #'learning_rate': hp.uniform('learning_rate', 0.00001, 0.01),
}


# 静止状態

df = pd.read_csv('KCB2.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:,:6]
y_train = df.iloc[:,6]
df2 = pd.read_csv('okada_K2.csv')
df2 = df2.sample(frac=1, random_state=0)
X_test = df2.iloc[:,:6]
y_test = df2.iloc[:,6]


def function(args):
    #n_hidden = int(args['n_hidden'])
    n_hidden = args['n_hidden']
    n_drop = args['n_drop']
    batch_size = int(args['batch_size'])
    optimizer = args['optimizer']
    n_neurons = args['n_neurons']
    print(args['n_hidden'],args['n_neurons'],args['n_drop'],args['batch_size'],args['optimizer'])
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(6,)))
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons[layer], activation='relu'))
        model.add(layers.Dropout(n_drop[layer]))
        #print(args['nneurons'][layer],args['ndrop'][layer])
    model.add(layers.Dense(1, activation='linear'))
    #model.summary()
    
    model.compile(optimizer=optimizer, loss='mse', metrics='mae')

    callbacks = [EarlyStopping(monitor = 'val_mae', patience = 20)]
    
    model.fit(X_train,
              y_train,
              epochs=1000,
              batch_size=batch_size,
              verbose=0,
              callbacks=callbacks,
              validation_split=0.1) # 検証用のデータ
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    return evaluation[0]

#for i in tqdm(range(200)):
 #   trials = Trials()
  #  best = fmin(function,
   #             hyperopt_parameters,
    #            algo=tpe.suggest,
     #           max_evals=1,
      #          trials=trials,
       #         verbose=0)
    
trials = Trials()
best = fmin(function,
            hyperopt_parameters,
            algo=tpe.suggest,
            max_evals=200,
            trials=trials,
            verbose=0)

In [5]:
print(best)

{'batch_size': 0, 'epochs': 2, 'l1_drop': 0.14296488874197943, 'l1_out': 3, 'l2_drop': 0.1662930115272234, 'l2_out': 5, 'optimizer': 2}


# GP-EIによる最適化

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from GPyOpt.methods import BayesianOptimization
from tqdm.notebook import tqdm

# 静止状態
df = pd.read_csv('KCB2.csv')
df = df.sample(frac=1, random_state=0)
X_train = df.iloc[:,:6]
y_train = df.iloc[:,6]
df2 = pd.read_csv('okada_K2.csv')
df2 = df2.sample(frac=1, random_state=0)
X_test = df2.iloc[:,:6]
y_test = df2.iloc[:,6]

# 目的関数の定義
def function(parameters):
    #print(parameters)
    n_hidden = int(parameters[0, 0])
    #n_drop = parameters[0, 1]
    batch_size = int(parameters[0, 1])
    optimizer = ['adam', 'sgd', 'rmsprop'][int(parameters[0, 2])]
    #n_neurons = parameters[0, 4:n_hidden+4].astype(int)
    n_neurons = parameters[0, 3:13].astype(int)
    n_drop = parameters[0, 13:23]
   # print(n_hidden,n_neurons, n_drop,batch_size,optimizer)
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(6,)))
    for i in range(n_hidden):
        model.add(layers.Dense(n_neurons[i], activation='relu'))
        model.add(layers.Dropout(n_drop[i]))
    model.add(layers.Dense(1, activation='linear'))

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae'])

    callbacks = [EarlyStopping(monitor='val_mae', patience=20)]

    history = model.fit(x=X_train,
                        y=y_train,
                        epochs=2000,
                        batch_size=batch_size,
                        verbose=0,
                        callbacks=callbacks,
                        validation_split=0.1)
    
    evaluation = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print(evaluation[0])
    return evaluation[0]
    #score = model.evaluate(X_test, y_test)
    #predictions = predict_function(model, X_test).flatten()
    #predictions = model.predict(X_test).flatten()
    #mse = mean_squared_error(y_test, predictions)
    #return -mse

# ハイパーパラメータの範囲とタイプの指定
#max_layers = 10  # チューニングする最大層数
bounds = [{'name': 'n_hidden', 'type': 'discrete', 'domain': (1, 2, 3, 4, 5)},
          #{'name': 'n_drop', 'type': 'continuous', 'domain': (0.3, 0.5)},
          {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32, 64)},
          {'name': 'optimizer', 'type': 'discrete', 'domain': (0, 1, 2)}]
for i in range(1, 11):
    bounds += [{'name': 'n_neurons_{}'.format(i), 'type': 'discrete', 'domain': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}]
for j in range(1, 11):
    bounds += [{'name': 'n_drop_{}'.format(j), 'type': 'continuous', 'domain': (0.3, 0.5)}]
# ベイズ最適化の設定
myRelt = BayesianOptimization(f=function,
                              domain=bounds,
                              #initinal_design_numdata=5,
                              acquisition_type='EI')
myRelt.run_optimization(max_iter=200)

# チューニング結果の取得
X = myRelt.X  # パラメータの履歴
Y = myRelt.Y  # 目的関数の値の履歴

print(myRelt.x_opt)     #最適なハイパーパラメータ
print(myRelt.fx_opt)    #MSE

#myRelt.plot_acquisition()
#myRelt.plot_convergence()


###最適なハイパーパラメータとそのときのMSEを表示
#best_params = myRelt.X[np.argmin(myRelt.Y)]
#best_mse = -np.min(myRelt.Y)
#print("Best Parameters:", best_params)
#print("Best MSE:", best_mse)